# Power output for REZ for different energy scenarios

In [147]:
from dask.distributed import Client,LocalCluster
from dask_jobqueue import PBSCluster

In [148]:
# One node on Gadi has 48 cores - try and use up a full node before going to multiple nodes (jobs)

walltime = "00:30:00"
cores = 24
memory = str(4 * cores) + "GB"

cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory), processes=cores,
                     job_extra_directives=["-q normal",
                                           "-P w42",
                                           "-l ncpus="+str(cores),
                                           "-l mem="+str(memory),
                                           "-l storage=gdata/w42+gdata/rt52"],
                     local_directory="$TMPDIR",
                     job_directives_skip=["select"])

In [149]:
cluster.scale(jobs=1)
client = Client(cluster)

In [150]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.121.7:43987,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [1]:
import xarray as xr
import pandas as pd

In [104]:
%cd /g/data/w42/dr6273/work/power_models
import functions as fn

/g/data/w42/dr6273/work/power_models


In [105]:
%load_ext autoreload
%autoreload 2

#### REZ mask

In [2]:
mask = xr.open_dataset('/g/data/w42/dr6273/work/projects/Aus_energy/data/rez_2024_mask_era5_grid.nc')

In [18]:
mask

<xarray.Dataset>
Dimensions:      (lat: 141, lon: 181, region: 43)
Coordinates:
  * lat          (lat) float32 -10.0 -10.25 -10.5 -10.75 ... -44.5 -44.75 -45.0
  * lon          (lon) float32 110.0 110.2 110.5 110.8 ... 154.5 154.8 155.0
  * region       (region) object 'Q1' 'Q2' 'Q3' 'Q4' ... 'T1' 'T2' 'T3' 'T4'
    region_code  (region) int64 ...
    latitude     (region) float64 ...
    longitude    (region) float64 ...
Data variables:
    REZ          (region, lat, lon) bool ...

#### REZ generation

In [6]:
gen = pd.read_csv("/g/data/w42/dr6273/work/data/REZ/2024/REZ_potential.csv", index_col=0)

In [14]:
# Tidy column names
gen.columns = [i.replace("90", "9-") for i in gen.columns]

In [19]:
gen.head()

,Solar renewable potential (MW),Wind renewable potential (MW),Existing solar,Progressive solar 2029-30,Progressive solar 2039-40,Progressive solar 2049-50,Step change solar 2029-30,Step change solar 2039-40,Step change solar 2049-50,Green energy solar 2029-30,...,Existing wind,Progressive wind 2029-30,Progressive wind 2039-40,Progressive wind 2049-50,Step change wind 2029-30,Step change wind 2039-40,Step change wind 2049-50,Green energy wind 2029-30,Green energy wind 2039-40,Green energy wind 2049-50
REZ,,,,,,,,,,,,,,,,,,,,,
N1,6385,0,166,50,50,100,0,50,100,50,...,0,0,0,0,0,0,0,0,0,-300
N2,2950,7400,855,50,500,3000,0,2400,5400,2300,...,442,3050,3600,7400,3000,7400,7400,4550,8700,9150
N3,6850,3000,1497,1700,1950,6100,1450,4400,6850,2950,...,673,3700,3700,4200,5450,5450,7800,6700,6800,7500
N4,8000,5100,53,200,200,350,150,150,400,200,...,198,100,100,100,100,100,150,150,150,150
N5,2256,3900,1122,700,700,2250,1150,1150,2250,800,...,0,450,450,450,600,600,1000,1000,1000,1000


In [81]:
def get_scaled_capacity_mask(mask_da, generation):
    """
    Return DataArray with mask of capacity divided evenly across grid cells
    
    mask_da: array of REZ mask
    generation: pandas Series of capacity values for each region (index).
    """
    da_list = []
    for r in generation.index:
        capacity = generation.loc[r]
        n_cells = mask_da.sel(region=r).sum().values
        
        da = mask_da.sel(region=r).where(
            mask_da.sel(region=r) == 0,
            capacity / n_cells
        ).expand_dims({"region": [r]})
        
        da_list.append(da)
        
    return xr.concat(da_list, dim="region")

In [203]:
def calc_generation(capacity_factor, generation_capacity):
    """
    Return DataArray with time series of power for each REZ
    
    capacity_factor: DataArray of capacity factors
    generation_capacity: DataArray of each grid cells generation capacity
    """
    da_list = []
    for r in generation_capacity.region.values:
        r_mask = generation_capacity.sel(region=r).where(
            generation_capacity.sel(region=r) > 0, drop=True
        )
        da = capacity_factor * r_mask
        da = da.sum(["lat", "lon"])
        da = da.expand_dims({"region": [r]})

        da_list.append(da)

    return xr.concat(da_list, dim="region")

In [151]:
sc2030_wind = get_scaled_capacity_mask(mask.REZ, gen["Step change wind 2029-30"])

In [171]:
cf = fn.load_hourly_cf(1960, "wind")["capacity_factor"]

In [204]:
da = calc_generation(cf, sc2030_wind)

In [205]:
da.sel(region="N4").isel(time=10).compute()

<xarray.DataArray ()>
array(34.19374734)
Coordinates:
    region       <U2 'N4'
    time         datetime64[ns] 1960-01-01T10:00:00
    region_code  int64 3
    latitude     float64 -31.97
    longitude    float64 141.4